# 0 Init

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/NUS_MSBA/CS5344/CS5344 project/dataset

/content/drive/MyDrive/NUS_MSBA/CS5344/CS5344 project/dataset


In [3]:
!pip install mlxtend

In [4]:
import csv
import os
import json
import argparse
import random
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter



from tqdm import tqdm

# # spelling
# from spellchecker import SpellChecker

# nltk
import nltk
from nltk.corpus import stopwords

# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

# # language detector
# from langdetect import detect
# from langdetect import DetectorFactory
# DetectorFactory.seed = 0

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 'here_for,'smoke','drink', 'match_age'
union_meta = ['scam','username','age','gender','location','ethnicity','occupation','marital_status','children','religion','sexual_orientation','sex', 'description']

scam_meta = ['email']

tqdm.pandas()
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
all_profile = pd.read_csv("processed_data.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2 Market Basket for Attributes

In [6]:
attribute_col = ['username','scam','age_bin','gender','location','ethnicity','marital_status','children','religion','sexual_orientation']
basket_col = ['scam','age_bin','gender','location','ethnicity','marital_status','children','religion','sexual_orientation']

all_profile_attribute = all_profile[attribute_col]
all_profile_attribute.dropna(subset=['username'], inplace=True)
all_profile_attribute.set_index('username', inplace=True)

# scam and non-scam
all_profile_attribute_scam = all_profile_attribute[all_profile_attribute['scam'] == 1]
all_profile_attribute_nonscam = all_profile_attribute[all_profile_attribute['scam'] == 0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-6-9e542e687be6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_profile_attribute.dropna(subset=['username'], inplace=True)


In [7]:
# change column value into "col name - value" format
def transform_row(row):
    return [f"{column} - {value}" for column, value in row.items() if pd.notna(value)]

all_profile_attribute_scam_b = all_profile_attribute_scam[basket_col].apply(transform_row, axis=1)
all_profile_attribute_nonscam_b = all_profile_attribute_nonscam[basket_col].apply(transform_row, axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
all_profile_attribute_scam_b

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


username
kaylabrandy19          [scam - 1, age_bin - 31-40, gender - female, l...
queenpam288            [scam - 1, age_bin - 21-30, gender - female, l...
victoriaia             [scam - 1, age_bin - 41-50, gender - female, l...
harlimosely            [scam - 1, age_bin - 31-40, gender - female, l...
martha1010             [scam - 1, age_bin - 31-40, gender - female, m...
                                             ...                        
ryandad8               [scam - 1, age_bin - 41-50, gender - male, loc...
marcusw670             [scam - 1, age_bin - 51-60, gender - male, loc...
marcusw670             [scam - 1, age_bin - 51-60, gender - male, loc...
lovelessmichaelklus    [scam - 1, age_bin - 41-50, gender - male, loc...
lovelessmichaelklus    [scam - 1, age_bin - 41-50, gender - male, loc...
Length: 6064, dtype: object

In [9]:
def onehot_enc(df):
  encoder = TransactionEncoder().fit(df)
  onehot = encoder.transform(df)
  df = pd.DataFrame(onehot, columns = encoder.columns_)
  return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
all_profile_attribute_scam_enc = onehot_enc(all_profile_attribute_scam_b)
all_profile_attribute_nonscam_enc = onehot_enc(all_profile_attribute_nonscam_b)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Run Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(all_profile_attribute_scam_enc, min_support=0.2, use_colnames=True)

# Generate association rules
scam_rules_raw = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
scam_rules_raw = scam_rules_raw[scam_rules_raw['consequents'] == {'scam - 1'}]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Run Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(all_profile_attribute_nonscam_enc, min_support=0.2, use_colnames=True)

# Generate association rules
nonscam_rules_raw = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
nonscam_rules_raw = nonscam_rules_raw[nonscam_rules_raw['consequents'] == {'scam - 0'}]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# remove duplicates
scam_antecedents_set = set(scam_rules_raw['antecedents'])
nonscam_antecedents_set = set(nonscam_rules_raw['antecedents'])

scam_antecedents_set_processed = scam_antecedents_set - nonscam_antecedents_set
nonscam_antecedents_set_processed = nonscam_antecedents_set - scam_antecedents_set

scam_rules = scam_rules_raw[scam_rules_raw['antecedents'].isin(scam_antecedents_set_processed)]
nonscam_rules = nonscam_rules_raw[nonscam_rules_raw['antecedents'].isin(nonscam_antecedents_set_processed)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
scam_rules_top7 = scam_rules[['antecedents', 'consequents', 'support']].sort_values(by='support', ascending=False)
scam_rules_top7

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support
50,"(gender - male, age_bin - 51-60)",(scam - 1),0.283146
26,(marital_status - widowed),(scam - 1),0.269129
78,"(gender - male, marital_status - widowed)",(scam - 1),0.248516
62,"(gender - female, ethnicity - white)",(scam - 1),0.240600
38,"(age_bin - 31-40, gender - female)",(scam - 1),0.236313
84,"(marital_status - single, gender - female, age...",(scam - 1),0.210257
98,"(marital_status - single, gender - female, eth...",(scam - 1),0.209598


In [67]:
nonscam_rules_top7 = nonscam_rules[['antecedents', 'consequents', 'support']].sort_values(by='support', ascending=False)[:7]
nonscam_rules_top7

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support
206,"(gender - male, sexual_orientation - straight)",(scam - 0),0.616261
217,"(sexual_orientation - straight, marital_status...",(scam - 0),0.584152
223,"(religion - christian, sexual_orientation - st...",(scam - 0),0.489592
17,(children - no children),(scam - 0),0.413020
191,"(gender - male, marital_status - single)",(scam - 0),0.410625
25,(ethnicity - hispanic),(scam - 0),0.407639
364,"(gender - male, marital_status - single, sexua...",(scam - 0),0.394383


In [68]:
scam_rules_top7 = [set(x) for x in scam_rules_top7['antecedents']]
nonscam_rules_top7 = [set(x) for x in nonscam_rules_top7['antecedents']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 4 Validation

In [112]:
seed_value = 99999
np.random.seed(seed_value)

sample_size = 500

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [113]:
samples_scam = pd.DataFrame(all_profile_attribute_scam_b.sample(n=sample_size, random_state=seed_value))
samples_nonscam = pd.DataFrame(all_profile_attribute_nonscam_b.sample(n=sample_size, random_state=seed_value))
samples_scam

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
username,
brownmikel613,"[scam - 1, age_bin - 51-60, gender - male, loc..."
wendy5436,"[scam - 1, age_bin - 21-30, gender - female, e..."
castle,"[scam - 1, age_bin - 51-60, gender - male, loc..."
edenlover,"[scam - 1, age_bin - 51-60, gender - male, loc..."
mrjohn,"[scam - 1, age_bin - 41-50, gender - male, loc..."
...,...
briget98777,"[scam - 1, age_bin - 21-30, gender - female, l..."
realelena,"[scam - 1, age_bin - 31-40, gender - female, l..."
honeygraceful,"[scam - 1, age_bin - 21-30, gender - female, l..."


In [114]:
def calculate_marks(samples_df):
    # Initialize a new column for marks
    samples_df['marks'] = 0

    # Iterate over each sample
    for index, row in samples_df.iterrows():
        sample = set(row[0])

        # Iterate over each scam rule, if sample set contains rule set, then -5 score
        for scam_rule in scam_rules_top7:
            l = len(scam_rule)
            if scam_rule <= sample:
                samples_df.at[index, 'marks'] -= 0.5*l

        # Iterate over each nonscam rule, if sample set contains rule set, then +5 score
        for nonscam_rule in nonscam_rules_top7:
            l = len(nonscam_rule)
            if nonscam_rule <= sample:
                samples_df.at[index, 'marks'] += 0.5*l

    return samples_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Assuming scam_rules_top7_set and nonscam_rules_top7_set are your rule sets
samples_scam_scored = calculate_marks(samples_scam)
samples_nonscam_scored = calculate_marks(samples_nonscam)

score_distribution_scam = samples_scam['marks']
score_distribution_nonscam = samples_nonscam['marks']

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot the score distribution for scam samples
axes[0].hist(score_distribution_scam, bins=10, edgecolor='black')
axes[0].set_title('Scam Samples')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Frequency')

# Plot the score distribution for non-scam samples
axes[1].hist(score_distribution_nonscam, bins=10, edgecolor='black')
axes[1].set_title('Non-Scam Samples')
axes[1].set_xlabel('Score')
axes[1].set_ylabel('Frequency')

# Adjust layout to prevent clipping of titles
plt.tight_layout()

# Show the plot
plt.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming samples_scam and samples_nonscam are your DataFrames with the 'marks' column

# Add 'actual_class' column based on whether the sample is from samples_scam
samples_scam['actual_class'] = 'scam'
samples_nonscam['actual_class'] = 'non-scam'

# Combine scam and non-scam sets
combined_samples = pd.concat([samples_scam, samples_nonscam], ignore_index=True)

# Filter out samples with a score of 0
combined_samples_filtered = combined_samples[combined_samples['marks'] != 0]

# Predicted class based on marks
combined_samples_filtered['predicted_class'] = combined_samples_filtered['marks'].apply(lambda x: 'scam' if x < 0 else 'non-scam')

# Create a confusion matrix
conf_matrix = confusion_matrix(combined_samples_filtered['actual_class'], combined_samples_filtered['predicted_class'], labels=['scam', 'non-scam'])

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['scam', 'non-scam'], yticklabels=['scam', 'non-scam'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()

In [ ]:
# Calculate percentage of samples with score != 0 for each dataset
percentage_not_zero_combined = (combined_samples_filtered.shape[0] / combined_samples.shape[0]) * 100
percentage_not_zero_scam = (samples_scam[samples_scam['marks'] != 0].shape[0] / samples_scam.shape[0]) * 100
percentage_not_zero_nonscam = (samples_nonscam[samples_nonscam['marks'] != 0].shape[0] / samples_nonscam.shape[0]) * 100

print(f"Percentage of samples with non-zero score in combined samples: {percentage_not_zero_combined:.2f}%")
print(f"Percentage of samples with non-zero score in scam samples: {percentage_not_zero_scam:.2f}%")
print(f"Percentage of samples with non-zero score in non-scam samples: {percentage_not_zero_nonscam:.2f}%")

# Calculate percentage of correct predictions for samples with score != 0
correct_predictions_combined = np.diag(conf_matrix).sum() / conf_matrix.sum() * 100
correct_predictions_scam = conf_matrix[0, 0] / conf_matrix[0, :].sum() * 100
correct_predictions_nonscam = conf_matrix[1, 1] / conf_matrix[1, :].sum() * 100

print(f"Percentage of correct predictions in combined samples with non-zero score: {correct_predictions_combined:.2f}%")
print(f"Percentage of correct predictions in scam samples with non-zero score: {correct_predictions_scam:.2f}%")
print(f"Percentage of correct predictions in non-scam samples with non-zero score: {correct_predictions_nonscam:.2f}%")